In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import MinMaxScaler

In [14]:
def plot_imputation_results(real_dataset, generated_dataset, mask, startindex, endindex, path):

    tr = real_dataset[startindex:endindex, 1]
    te = generated_dataset[startindex:endindex, 1]
    mask1 = mask[startindex:endindex, 1]
    te = (mask1 * te) + (1 - mask1) * tr
    mlt.figure(figsize=(20, 6))
    y = np.arange(1, len(tr) + 1)

    start_idx = 0
    for i in range(len(te)-1):
        color = "green"
        mlt.plot(y[start_idx:i+2],te[start_idx:i+2], color=color, linewidth = '3')
        start_idx = i + 1

    start_idx = 0
    for i in range(len(tr)-1):
        color = "blue"
        if mask1[i] == 0:
            mlt.plot(y[start_idx:i+2],tr[start_idx:i+2], color=color)
        elif mask1[i] == 1:
            color = 'yellow'
            mlt.plot(y[start_idx:i+2],tr[start_idx:i+2], color=color, linestyle='--')
        start_idx = i + 1


    mlt.suptitle('Gan prediction on training dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [1]:
def plot_full_dataset(real_dataset, gen_dataset, startindex, endindex, path, col):
    tr = real_dataset[startindex:endindex, col]
    te = gen_dataset[startindex:endindex, col]
    mlt.figure(figsize=(20, 6))
    mlt.suptitle('Gan prediction on test dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.plot(tr, label='real')
    mlt.plot(te, label='gen')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [4]:
def compare_predicted_dataset(real_dataset, gen_dataset_mogrifier, gen_dataset_bilstm, gen_dataset_lstm, startindex, endindex, path, col):
    tr = real_dataset[startindex:endindex, col]
    te1 = gen_dataset_mogrifier[startindex:endindex, col]
    te2 = gen_dataset_bilstm[startindex:endindex, col]
    te3 = gen_dataset_lstm[startindex:endindex, col]
    mlt.figure(figsize=(20, 10))
    mlt.suptitle('Gan prediction on test dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.plot(tr, label='True value', marker='x')
    mlt.plot(te1, label='Mogrifier', marker='x')
    mlt.plot(te2, label='Bilstm', marker='x')
    mlt.plot(te3, label='lstm', marker='x')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [16]:
# df = pd.read_csv("Data/Processed/acn_data_with_zero.csv")

# plot_acf(df['Energy'], lags=100)  # Set the number of lags to display
# mlt.title('Autocorrelation Plot using statsmodels.graphics.tsaplots.plot_acf')
# mlt.show()

In [17]:
# df = pd.read_csv("Data/Processed/boulder_data_with_zero.csv")
# df = df.loc[(df['Start'] >= '2018-01-02 00:00:00') & (df['Start'] <= '2020-12-32 21:00:00')]
# df.drop(columns=['Start'], inplace=True)

# df2 = pd.read_csv("Data/Processed/acn_data_with_zero.csv")
# df2 = df2.loc[(df2['Start'] >= '2018-01-02 00:00:00') & (df2['Start'] <= '2020-12-32 21:00:00')]
# df2.drop(columns=['Start'], inplace=True)

# scaler = MinMaxScaler(feature_range=(0,1))
# df = scaler.fit_transform(df)
# df = pd.DataFrame(df, columns=['Day of week', 'Energy', 'Week Day', 'Year', 'Month', 'Day of month'])
# tr = df['Energy'][:50000].reset_index(drop=True)

# scaler = MinMaxScaler(feature_range=(0,1))
# df2 = scaler.fit_transform(df2)
# df2 = pd.DataFrame(df2, columns=['Day of week', 'Energy', 'Week Day', 'Year', 'Month', 'Day of month'])
# tr2 = df2['Energy'][:50000].reset_index(drop=True)

# mlt.figure(figsize=(20, 6))
# mlt.suptitle('Gan prediction on test dataset')
# mlt.ylabel('Energy Consumption in Kwh')
# mlt.plot(tr, label='Boulder')
# mlt.plot(tr2, label='Acn')
# # mlt.plot(tr3, label='Dundee')
# mlt.legend()
# mlt.show()

In [18]:
# df.describe()

In [19]:
# data1 = pd.read_csv("Data/Processed/acn_data_with_zero.csv")
# data2 = pd.read_csv("Data/Processed/palo_alto_data_with_zero.csv")
# data1 = data1.loc[data1['Start'] > '2018-01-01 23:00:00']
# data2 = data2.loc[(data2['Start'] <= '2018-01-01 23:00:00') & (data2['Start'] >= '2016-01-01 00:00:00')]
# df = pd.concat([data1, data2], axis=0).reset_index(drop=True)
# df['Sum'] = df.groupby(pd.to_datetime(df['Start']).dt.date)['Energy'].cumsum()
# df.drop(columns=['Start'], inplace=True)
# tr = df['Energy'][4000:4500]
# mlt.figure(figsize=(20, 6))
# mlt.suptitle('Gan prediction on test dataset')
# mlt.ylabel('Energy Consumption in Kwh')
# mlt.plot(tr, label='Boulder')
# # mlt.plot(tr3, label='Dundee')
# mlt.legend()
# mlt.show()